In [1]:
import pandas as pd
import numpy as np
import gc
import sys

In [2]:
def return_size(df):
    ## get dataframe size in GB
    return round(sys.getsizeof(df) / 1e9, 2)

def reduce_memory(df):
    original_memory = df.memory_usage().sum()
    
    # Iterate through columns
    for col in df:
        
        # ID to 32 bit integer
        if ('SK_ID' in col):
            df[col] = df[col].fillna(0).astype(np.int32)
            
        # objects to category
        elif (df[col].dtype == 'object') and (df[col].nunique() < df.shape[0]):
            df[col] = df[col].astype('category')
        
        # 1/0 integers to booleans
        elif list(df[col].unique()) == [1, 0]:
            df[col] = df[col].astype(bool)
        
        # Float64 to float32
        elif df[col].dtype == float:
            df[col] = df[col].astype(np.float32)
            
        # 64 bit integers to 32 bit
        elif df[col].dtype == int:
            df[col] = df[col].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    print('Original Memory Usage: {} GB.'.format(round(original_memory / 1e9, 2)))
    print('New Memory Usage: {} GB.'.format(round(new_memory / 1e9, 2)))
        
    return df

### Main dataset
The main dataset contains a unique credit identifiers SK_ID_CURR, 120 independent variables and a target variable indicating if the debtor defaulted on the credit.

In [3]:
df = pd.read_csv('./home-credit-default-risk/application_train.csv')
df_test = pd.read_csv('./home-credit-default-risk/application_test.csv')

df['test'] = False
df_test['test'] = True

df = df.append(df_test)
df = reduce_memory(df).copy()
print(df.test.value_counts())
df.head()

Original Memory Usage: 0.35 GB.
New Memory Usage: 0.15 GB.
False    307511
True      48744
Name: test, dtype: int64


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,test
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,False
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,False
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [4]:
gc.enable()
del df_test
gc.collect()

21

### Quick feature engineering
I repeat the feature engineering discussed in the first notebook. <br>
The false entries in *DAYS_EMPLOYED* are corrected and three custom ratios are created.

In [5]:
df['DAYS_EMPLOYED_flagged'] = df["DAYS_EMPLOYED"] == 365243
df.loc[df.DAYS_EMPLOYED_flagged == True, 'DAYS_EMPLOYED'] = np.nan

In [6]:
df['income_pct'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
df['annuity_pct'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['credit_term'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

## Supplementary data
I will now begin adding the supplementary data to my dataset. <br>
The supplementary data often times consists of multiple rows corresponding to one entry in the main dataset. I create two functions to aggregate values for each client.

In [7]:
def aggregate_num(df, group_by, prefix):
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_by and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    # Only want the numeric variables
    parent_ids = df[group_by].copy()
    numeric_df = df.select_dtypes('number').copy()
    numeric_df[group_by] = parent_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_by).agg(['count', 'mean', 'max', 'min', 'sum'])

    # Need to create new column names
    columns = []

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        if var != group_by:
            # Iterate through the stat names
            for stat in agg.columns.levels[1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (prefix, var, stat))
    
    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg

In [8]:
def aggregate_cat(df, group_by, prefix):
 
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('category'))

    # Make sure to put the identifying id on the column
    categorical[group_by] = df[group_by]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_by).agg(['sum', 'count', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['sum', 'count', 'mean']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (prefix, var, stat))
    
    categorical.columns = column_names
    
    # Remove duplicate columns by values
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

### Credit bureau
This dataset contains information from a credit bureau, showing information about large number of active and closed credits. Each credit can be linked to a credit in the current dataset via the *SK_ID_CURR* feature.

In [9]:
bureau = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau = reduce_memory(bureau)
bureau.head()

Original Memory Usage: 0.23 GB.
New Memory Usage: 0.1 GB.


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [10]:
bureau_num = aggregate_num(bureau, 'SK_ID_CURR', 'bureau')
bureau_num.head()

,bureau_DAYS_CREDIT_sum,bureau_DAYS_ENDDATE_FACT_sum,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_CREDIT_UPDATE_min,...,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_DEBT_sum,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-5145,-3302.0,-1572,-1329.0,-1328.0,-825.500000,-735.000000,-652,-544.0,-155,...,85240.929688,85500.0,207623.578125,373239.0,378000.000000,596686.5,1.453365e+06,NaN,NaN,NaN
100002,-6992,-4185.0,-1437,-1072.0,-1185.0,-697.500000,-874.000000,-3999,-36.0,-1185,...,49156.199219,0.0,108131.945312,245781.0,450000.000000,245781.0,8.650556e+05,0.0,1681.029053,5043.64502
100003,-5603,-3292.0,-2586,-2434.0,-2131.0,-1097.333374,-1400.750000,-3264,-540.0,-2131,...,0.000000,22248.0,254350.125000,0.0,810000.000000,0.0,1.017400e+06,0.0,0.000000,0.00000
100004,-1734,-1065.0,-1326,-595.0,-683.0,-532.500000,-867.000000,-1064,-382.0,-682,...,0.000000,94500.0,94518.898438,0.0,94537.796875,0.0,1.890378e+05,0.0,0.000000,0.00000
100005,-572,-123.0,-373,-128.0,-123.0,-123.000000,-190.666667,-163,-123.0,-121,...,189469.500000,29826.0,219042.000000,543087.0,568800.000000,568408.5,6.571260e+05,0.0,0.000000,0.00000


In [11]:
bureau_cat = aggregate_cat(bureau, 'SK_ID_CURR', 'bureau')
bureau_cat.head()

,bureau_CREDIT_TYPE_Mobile operator loan_mean,bureau_CREDIT_TYPE_Mobile operator loan_sum,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_mean,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,bureau_CREDIT_ACTIVE_Bad debt_mean,bureau_CREDIT_ACTIVE_Bad debt_sum,bureau_CREDIT_TYPE_Interbank credit_mean,bureau_CREDIT_TYPE_Interbank credit_sum,bureau_CREDIT_TYPE_Real estate loan_mean,bureau_CREDIT_TYPE_Real estate loan_sum,...,bureau_CREDIT_TYPE_Credit card_sum,bureau_CREDIT_ACTIVE_Active_mean,bureau_CREDIT_ACTIVE_Closed_mean,bureau_CREDIT_TYPE_Consumer credit_mean,bureau_CREDIT_CURRENCY_currency 1_mean,bureau_CREDIT_ACTIVE_Active_sum,bureau_CREDIT_ACTIVE_Closed_sum,bureau_CREDIT_TYPE_Consumer credit_sum,bureau_CREDIT_CURRENCY_currency 1_sum,bureau_CREDIT_ACTIVE_Active_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.428571,0.571429,1.000000,1.0,3,4,7,7,7
100002,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,4,0.250000,0.750000,0.500000,1.0,2,6,4,8,8
100003,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,2,0.250000,0.750000,0.500000,1.0,1,3,2,4,4
100004,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.000000,1.000000,1.000000,1.0,0,2,2,2,2
100005,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1,0.666667,0.333333,0.666667,1.0,2,1,2,3,3


In [12]:
df = df.merge(bureau_cat, on = 'SK_ID_CURR', how = 'left')
df = df.merge(bureau_num, on = 'SK_ID_CURR', how = 'left')
print(len(df.columns))

230


### Credit bureau balance
This dataset shows the monthly balance of each client in the credit bureau. The *SK_ID_BUREAU* feature can be linked to *SK_ID_CURR* via the previous dataset.

In [13]:
bureau_balance = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
bureau_balance = reduce_memory(bureau_balance)
bureau_balance.head()

Original Memory Usage: 0.66 GB.
New Memory Usage: 0.25 GB.


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [14]:
bureau_balance_cat = aggregate_cat(bureau_balance, 'SK_ID_BUREAU', 'bureau_balance')
bureau_balance_num = aggregate_num(bureau_balance, 'SK_ID_BUREAU', 'bureau_balance')

# Dataframe grouped by loan
bureau_balance_loans = bureau_balance_num.merge(bureau_balance_cat, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

# Merge to include the SK_ID_CURR
bureau_balance_loans = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_balance_loans, on = 'SK_ID_BUREAU', how = 'right')

# Aggregate the stats for each client
bureau_balance_clients = aggregate_num(bureau_balance_loans.drop(columns = ['SK_ID_BUREAU']), 'SK_ID_CURR', 'client')

bureau_balance_clients.head()

,client_bureau_balance_MONTHS_BALANCE_sum_sum,client_bureau_balance_MONTHS_BALANCE_sum_min,client_bureau_balance_MONTHS_BALANCE_sum_mean,client_bureau_balance_MONTHS_BALANCE_min_sum,client_bureau_balance_MONTHS_BALANCE_mean_sum,client_bureau_balance_MONTHS_BALANCE_min_min,client_bureau_balance_MONTHS_BALANCE_mean_min,client_bureau_balance_MONTHS_BALANCE_min_mean,client_bureau_balance_MONTHS_BALANCE_mean_mean,client_bureau_balance_MONTHS_BALANCE_sum_max,...,client_bureau_balance_STATUS_X_sum_max,client_bureau_balance_STATUS_0_sum_max,client_bureau_balance_STATUS_C_sum_mean,client_bureau_balance_MONTHS_BALANCE_count_mean,client_bureau_balance_STATUS_X_sum_sum,client_bureau_balance_STATUS_0_sum_sum,client_bureau_balance_STATUS_C_sum_max,client_bureau_balance_MONTHS_BALANCE_count_max,client_bureau_balance_STATUS_C_sum_sum,client_bureau_balance_MONTHS_BALANCE_count_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,-2800,-1326,-400.000000,-165,-82.5,-51,-25.5,-23.571429,-11.785714,-1,...,9,12,15.714286,24.571429,30.0,31.0,44,52,110.0,172
100002.0,-2701,-632,-337.625000,-226,-175.0,-47,-39.5,-28.250000,-21.875000,-6,...,3,18,2.875000,13.750000,15.0,45.0,13,22,23.0,110
100005.0,-91,-78,-30.333333,-18,-9.0,-12,-6.0,-6.000000,-3.000000,-3,...,1,7,1.666667,7.000000,2.0,14.0,5,13,5.0,21
100010.0,-3312,-2610,-1656.000000,-127,-92.0,-90,-72.5,-63.500000,-46.000000,-702,...,0,10,26.000000,36.000000,0.0,20.0,26,36,52.0,72
100013.0,-6756,-2346,-1689.000000,-226,-113.0,-68,-34.0,-56.500000,-28.250000,-780,...,40,34,25.750000,57.500000,41.0,79.0,44,69,103.0,230


In [15]:
df = df.merge(bureau_cat, on = 'SK_ID_CURR', how = 'left')
df = df.merge(bureau_num, on = 'SK_ID_CURR', how = 'left')
df = df.merge(bureau_balance_clients, on = 'SK_ID_CURR', how = 'left')
print(len(df.columns))

418


In [16]:
gc.enable()
del bureau, bureau_num, bureau_cat, bureau_balance, bureau_balance_cat, bureau_balance_num, bureau_balance_loans, bureau_balance_clients
gc.collect()

21

### Previous applications
This dataset contains information about loans previously given to clients by the institution of this competition.

In [17]:
previous = pd.read_csv('./home-credit-default-risk/previous_application.csv')
previous = reduce_memory(previous)
previous.head()

Original Memory Usage: 0.49 GB.
New Memory Usage: 0.16 GB.


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430054,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615234,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735352,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335938,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.394531,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
previous_num = aggregate_num(previous, 'SK_ID_CURR', 'previous')
previous_num.head()

,previous_DAYS_DECISION_sum,previous_DAYS_DECISION_min,previous_DAYS_DECISION_mean,previous_DAYS_DECISION_max,previous_DAYS_FIRST_DUE_sum,previous_DAYS_FIRST_DUE_min,previous_DAYS_FIRST_DUE_mean,previous_DAYS_FIRST_DUE_max,previous_DAYS_LAST_DUE_sum,previous_DAYS_LAST_DUE_min,...,previous_DAYS_FIRST_DRAWING_min,previous_DAYS_FIRST_DRAWING_mean,previous_DAYS_FIRST_DRAWING_max,previous_DAYS_FIRST_DRAWING_sum,previous_RATE_INTEREST_PRIMARY_min,previous_RATE_INTEREST_PRIMARY_mean,previous_RATE_INTEREST_PRIMARY_max,previous_RATE_INTEREST_PRIVILEGED_min,previous_RATE_INTEREST_PRIVILEGED_mean,previous_RATE_INTEREST_PRIVILEGED_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1740,-1740,-1740.0,-1740,-1709.0,-1709.0,-1709.000000,-1709.0,-1619.0,-1619.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
100002,-606,-606,-606.0,-606,-565.0,-565.0,-565.000000,-565.0,-25.0,-25.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
100003,-3915,-2341,-1305.0,-746,-3823.0,-2310.0,-1274.333374,-716.0,-3163.0,-1980.0,...,365243.0,365243.0,365243.0,1095729.0,NaN,NaN,NaN,NaN,NaN,NaN
100004,-815,-815,-815.0,-815,-784.0,-784.0,-784.000000,-784.0,-724.0,-724.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
100005,-1072,-757,-536.0,-315,-706.0,-706.0,-706.000000,-706.0,-466.0,-466.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
previous_cat = aggregate_cat(previous, 'SK_ID_CURR', 'previous')
previous_cat.head()

,previous_NAME_GOODS_CATEGORY_Animals_mean,previous_NAME_GOODS_CATEGORY_Animals_sum,previous_NAME_GOODS_CATEGORY_House Construction_mean,previous_NAME_GOODS_CATEGORY_House Construction_sum,previous_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_mean,previous_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_sum,previous_NAME_CASH_LOAN_PURPOSE_Money for a third person_mean,previous_NAME_CASH_LOAN_PURPOSE_Money for a third person_sum,previous_NAME_CASH_LOAN_PURPOSE_Hobby_mean,previous_NAME_CASH_LOAN_PURPOSE_Hobby_sum,...,previous_CODE_REJECT_REASON_XAP_mean,previous_FLAG_LAST_APPL_PER_CONTRACT_Y_mean,previous_NAME_PORTFOLIO_POS_sum,previous_NAME_CONTRACT_TYPE_Consumer loans_sum,previous_NAME_CASH_LOAN_PURPOSE_XAP_sum,previous_NAME_PRODUCT_TYPE_XNA_sum,previous_NAME_CONTRACT_STATUS_Approved_sum,previous_CODE_REJECT_REASON_XAP_sum,previous_FLAG_LAST_APPL_PER_CONTRACT_Y_sum,previous_NAME_CONTRACT_TYPE_Cash loans_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,1,1,1,1,1
100002,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,1,1,1,1,1
100003,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,2,2,2,2,3,3,3,3
100004,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,1,1,1,1,1
100005,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,2,1,2,2,2


In [20]:
df = df.merge(previous_num, on = 'SK_ID_CURR', how = 'left')
df = df.merge(previous_cat, on = 'SK_ID_CURR', how = 'left')
print(len(df.columns))

783


In [21]:
gc.enable()
del previous, previous_num, previous_cat
gc.collect()

21

### Monthly cash balance

In [22]:
def aggregate_client(df, group_vars, df_names):
    
    # aggregate numeric features
    df_num = aggregate_num(df, group_vars[0], df_names[0])
    
    # If there are categorical variables
    if any(df.dtypes == 'category'):
    
        # aggregate categorical features
        df_cat = aggregate_cat(df, group_vars[0], df_names[0])
        
        # merge
        df_loans = df_cat.merge(df_num, on = group_vars[0], how = 'outer')
        
        gc.enable()
        del df_cat
        gc.collect()
        
    else:
        df_loans = df_num
  
        
    # Merge to get the client id in dataframe
    df_loans = df_loans.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')

    # Remove the loan id
    df_loans = df_loans.drop(columns = [group_vars[0]])

    # aggregate again for clients
    df_clients = aggregate_num(df_loans, group_vars[1], df_names[1])
    
    
    gc.enable()
    del df_num, df_loans
    gc.collect()


    return df_clients

In [23]:
cash = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
cash = reduce_memory(cash)
cash.head()

Original Memory Usage: 0.64 GB.
New Memory Usage: 0.29 GB.


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [24]:
cash_balance = aggregate_client(cash, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])
cash_balance.head()

,client_cash_MONTHS_BALANCE_sum_sum,client_cash_MONTHS_BALANCE_min_sum,client_cash_MONTHS_BALANCE_mean_sum,client_cash_MONTHS_BALANCE_max_sum,client_cash_MONTHS_BALANCE_sum_min,client_cash_MONTHS_BALANCE_sum_mean,client_cash_MONTHS_BALANCE_sum_max,client_cash_MONTHS_BALANCE_min_min,client_cash_MONTHS_BALANCE_mean_min,client_cash_MONTHS_BALANCE_max_min,...,client_cash_CNT_INSTALMENT_FUTURE_max_sum,client_cash_NAME_CONTRACT_STATUS_Active_sum_sum,client_cash_CNT_INSTALMENT_min_sum,client_cash_CNT_INSTALMENT_mean_sum,client_cash_CNT_INSTALMENT_max_sum,client_cash_CNT_INSTALMENT_count_sum,client_cash_CNT_INSTALMENT_FUTURE_count_sum,client_cash_NAME_CONTRACT_STATUS_Active_count_sum,client_cash_CNT_INSTALMENT_FUTURE_sum_sum,client_cash_CNT_INSTALMENT_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-2887,-669,-653.0,-637,-378,-320.777778,-275,-96,-94.5,-93,...,28.0,32.0,36.0,36.000000,36.0,41,41,41,62.0,164.0
100002,-3610,-361,-190.0,-19,-190,-190.000000,-190,-19,-10.0,-1,...,456.0,361.0,456.0,456.000000,456.0,361,361,361,5415.0,8664.0
100003,-13240,-1348,-1226.0,-1104,-858,-472.857143,-172,-77,-71.5,-66,...,288.0,256.0,248.0,283.000000,288.0,272,272,272,1608.0,2840.0
100004,-408,-108,-102.0,-96,-102,-102.000000,-102,-27,-25.5,-24,...,16.0,12.0,12.0,15.000000,16.0,16,16,16,36.0,60.0
100005,-2420,-275,-220.0,-165,-220,-220.000000,-220,-25,-20.0,-15,...,132.0,99.0,99.0,128.699997,132.0,110,110,121,792.0,1287.0


In [25]:
df = df.merge(cash_balance, on = 'SK_ID_CURR', how = 'left')
print(len(df.columns))

945


In [26]:
gc.enable()
del cash, cash_balance
gc.collect()

21

### Credit card data

In [27]:
credit = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
credit = reduce_memory(credit)
credit.head()

Original Memory Usage: 0.71 GB.
New Memory Usage: 0.34 GB.


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [28]:
credit_agg = aggregate_client(credit, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['credit', 'client'])
credit_agg.head()

,client_credit_MONTHS_BALANCE_sum_sum,client_credit_MONTHS_BALANCE_min_sum,client_credit_MONTHS_BALANCE_mean_sum,client_credit_MONTHS_BALANCE_sum_min,client_credit_MONTHS_BALANCE_sum_mean,client_credit_MONTHS_BALANCE_sum_max,client_credit_MONTHS_BALANCE_max_sum,client_credit_MONTHS_BALANCE_min_min,client_credit_MONTHS_BALANCE_min_mean,client_credit_MONTHS_BALANCE_min_max,...,client_credit_AMT_DRAWINGS_ATM_CURRENT_mean_max,client_credit_AMT_PAYMENT_CURRENT_mean_min,client_credit_AMT_PAYMENT_CURRENT_mean_mean,client_credit_AMT_PAYMENT_CURRENT_mean_max,client_credit_AMT_PAYMENT_CURRENT_max_min,client_credit_AMT_PAYMENT_CURRENT_max_mean,client_credit_AMT_PAYMENT_CURRENT_max_max,client_credit_AMT_DRAWINGS_ATM_CURRENT_max_min,client_credit_AMT_DRAWINGS_ATM_CURRENT_max_mean,client_credit_AMT_DRAWINGS_ATM_CURRENT_max_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,-126,-36,-21.0,-21,-21.0,-21,-6,-6,-6.0,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100011,-210826,-5550,-2849.0,-2849,-2849.0,-2849,-148,-75,-75.0,-75,...,2432.432373,4843.063965,4843.063965,4843.063965,55485.0,55485.0,55485.0,180000.0,180000.0,180000.0
100013,-446976,-9216,-4656.0,-4656,-4656.0,-4656,-96,-96,-96.0,-96,...,6350.000000,7168.346191,7168.346191,7168.346191,153675.0,153675.0,153675.0,157500.0,157500.0,157500.0
100021,-2890,-306,-170.0,-170,-170.0,-170,-34,-18,-18.0,-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100023,-480,-88,-60.0,-60,-60.0,-60,-32,-11,-11.0,-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df = df.merge(credit_agg, on = 'SK_ID_CURR', how = 'left')
print(len(df.columns))

1321


In [30]:
gc.enable()
del credit, credit_agg
gc.collect()

21

### Installment payments

In [31]:
payment = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
payment = reduce_memory(payment)
payment.head()

Original Memory Usage: 0.87 GB.
New Memory Usage: 0.44 GB.


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [32]:
payment_agg = aggregate_client(payment, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['installments', 'client'])
payment_agg.head()

,client_installments_DAYS_ENTRY_PAYMENT_sum_sum,client_installments_DAYS_INSTALMENT_sum_sum,client_installments_DAYS_ENTRY_PAYMENT_min_sum,client_installments_DAYS_INSTALMENT_min_sum,client_installments_DAYS_ENTRY_PAYMENT_mean_sum,client_installments_DAYS_INSTALMENT_mean_sum,client_installments_DAYS_ENTRY_PAYMENT_max_sum,client_installments_DAYS_INSTALMENT_max_sum,client_installments_DAYS_INSTALMENT_sum_min,client_installments_DAYS_ENTRY_PAYMENT_sum_min,...,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_INSTALMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_max_sum,client_installments_AMT_PAYMENT_sum_sum,client_installments_AMT_INSTALMENT_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-52813.0,-52598.0,-15608.0,-15584.0,-15365.0,-15314.0,-15080.0,-15044.0,-8658.0,-8647.0,...,2.774678e+04,2.774678e+04,2.925090e+04,2.925090e+04,4.119593e+04,4.119593e+04,8.153775e+04,8.153775e+04,1.528387e+05,1.528387e+05
100002,-113867.0,-106495.0,-11153.0,-10735.0,-5993.0,-5605.0,-931.0,-475.0,-5605.0,-5993.0,...,1.757837e+05,1.757837e+05,2.196257e+05,2.196257e+05,2.196257e+05,2.196257e+05,1.008781e+06,1.008781e+06,4.172888e+06,4.172888e+06
100003,-367137.0,-365546.0,-37757.0,-37514.0,-34633.0,-34454.0,-31594.0,-31394.0,-25740.0,-25821.0,...,1.154108e+06,1.154108e+06,1.150977e+06,1.150977e+06,1.618865e+06,1.618865e+06,4.394102e+06,4.394102e+06,1.134881e+07,1.134881e+07
100004,-6855.0,-6786.0,-2385.0,-2352.0,-2285.0,-2262.0,-2181.0,-2172.0,-2262.0,-2285.0,...,1.607175e+04,1.607175e+04,2.128846e+04,2.128846e+04,2.128846e+04,2.128846e+04,3.172189e+04,3.172189e+04,6.386539e+04,6.386539e+04
100005,-49374.0,-47466.0,-6624.0,-6354.0,-5486.0,-5274.0,-4230.0,-4194.0,-5274.0,-5486.0,...,4.331880e+04,4.331880e+04,5.616184e+04,5.616184e+04,5.616184e+04,5.616184e+04,1.589062e+05,1.589062e+05,5.054566e+05,5.054566e+05


In [33]:
df = df.merge(payment_agg, on = 'SK_ID_CURR', how = 'left')
print(len(df.columns))

1427


In [34]:
gc.enable()
del payment, payment_agg
gc.collect()

21

In [35]:
df = reduce_memory(df)
df.head()

Original Memory Usage: 3.19 GB.
New Memory Usage: 2.01 GB.


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_INSTALMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_max_sum,client_installments_AMT_PAYMENT_sum_sum,client_installments_AMT_INSTALMENT_sum_sum
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1.757837e+05,1.757837e+05,2.196257e+05,2.196257e+05,2.196257e+05,2.196257e+05,1.008781e+06,1.008781e+06,4.172888e+06,4.172888e+06
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1.154108e+06,1.154108e+06,1.150977e+06,1.150977e+06,1.618865e+06,1.618865e+06,4.394102e+06,4.394102e+06,1.134881e+07,1.134881e+07
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,1.607175e+04,1.607175e+04,2.128846e+04,2.128846e+04,2.128846e+04,2.128846e+04,3.172189e+04,3.172189e+04,6.386539e+04,6.386539e+04
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,9.944767e+05,9.944767e+05,6.917869e+05,6.917869e+05,1.007153e+06,1.007153e+06,1.057860e+06,1.057860e+06,3.719995e+06,3.719995e+06
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,4.837564e+05,8.258458e+05,2.801997e+05,2.946311e+05,8.061279e+05,8.359853e+05,8.367034e+05,8.367034e+05,1.139621e+07,1.179154e+07


In [37]:
df.to_csv('dataset_full.csv', index = False, chunksize = 1000)